In [2]:
import cv2 
import imutils 
import numpy as np 
import pytesseract 
import requests 
import json 
import xmltodict 
import re 
import string 

In [3]:
vimal_model = cv2.face.LBPHFaceRecognizer_create()
vimal_model.read('lbph_car_detection.yml')

In [15]:
cap = cv2.VideoCapture("./car5.mp4")
fcount=0
while True:
    ret, frames = cap.read()
    gray = cv2.cvtColor(frames, cv2.COLOR_BGR2GRAY)
    results = vimal_model.predict(gray)
    if results[1] <500:
        confidence = int( 100 * (1 - (results[1])/400) )
    if confidence>= 85:
        fcount =fcount+1
    if fcount == 22:
        cv2.imwrite("photo2.jpeg",gray)
        print("Car Detected")
        break
    else:
        pass
cv2.imshow('photo2.jpeg',gray)
cv2.waitKey()=='13'
cv2.destroyAllWindows()
cap.release()

Car Detected


In [16]:
table = str.maketrans('', '', string.ascii_lowercase)

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
img = cv2.imread('photo2.jpeg',cv2.IMREAD_COLOR)
img = cv2.resize(img, (500,600) )
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
gray = cv2.bilateralFilter(gray, 13, 15, 15) 
edged = cv2.Canny(gray, 30, 200) 
contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
screenCnt = None
for c in contours:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)
    if len(approx) == 4:
        screenCnt = approx
        break

if screenCnt is None:
    detected = 0
    print ("No contour detected")
else:
    detected = 1
    
if detected == 1:
    cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

mask = np.zeros(gray.shape,np.uint8)
new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
new_image = cv2.bitwise_and(img,img,mask=mask)

(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx+1, topy:bottomy+1]

text = pytesseract.image_to_string(Cropped, config='--psm 11')

img = cv2.resize(img,(500,800))
Cropped = cv2.resize(Cropped,(400,200))

text = re.sub('[^A-Za-z0-9]+', '', text)
s = text
result = s.translate(table)
final_text =result[-9:]

cv2.imshow('Cropped',Cropped)
cv2.imshow('car',img)


print("License Plate Recognition\n")
print("Detected license plate Number is:",final_text)
cv2.waitKey()== 13
cv2.waitKey()== 13 
cv2.destroyAllWindows()

License Plate Recognition

Detected license plate Number is: DL1CN7197
